In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import requests
import operator
from collections import Counter
from collections import defaultdict
import math
from traitlets.traitlets import default
import pickle

In [8]:
URL = "https://hub.snapshot.org/graphql"

#DAO space
SPACE = "space.eth"

In [ ]:
#diccionario para guardar la info de la DAO
all_dao_info = {}

def set_basic_info(dao):
  QUERY = f"""
  query {{
  space(id: "{dao}") {{
      name
      network
      members
      admins
      followersCount
  }}
  }}
  """
  community_info = requests.post(URL, json={'query': QUERY}).json()
  community_info = community_info["data"]["space"]
  all_dao_info["basic_info"] = community_info

set_basic_info(SPACE)
print(all_dao_info["basic_info"])

In [ ]:
#todos los votos y las addresses de sus votantes, con fecha

def get_all_votes(dao, date):
  QUERY_VOTES = f"""query Votes {{
    votes(
      where: {{space_in: "{dao}", created_gt: {date} }},
      first: 1000,
      orderBy: "created",
      orderDirection: asc
    ) {{
      id
      created
      voter
      vp
      choice
      proposal {{
          id
          title
      }}
    }}
  }}
  """
  return QUERY_VOTES

def set_votes_info(dao):
  fecha = 0
  list_dict_results = []
  while True:

    votes = requests.post(URL, json={'query': get_all_votes(dao, fecha)}).json()["data"]["votes"]
    list_dict_results += votes
    fecha = max(list(map(lambda x: x["created"], votes)))

    if len(votes) < 1000:
      break

  df_votes = pd.DataFrame.from_dict(list_dict_results, orient='columns')
  df_votes['created'] = pd.to_datetime(df_votes['created'], unit='s')
  df_votes['voter'] = df_votes['voter'].str.lower()
  df_votes["choice"] = df_votes["choice"].apply(lambda val: val[0] if (type(val) == list) else val)

  all_dao_info["df_votes"] = df_votes


set_votes_info(SPACE)
#print(all_dao_info["df_votes"]["id"])



In [13]:
def get_all_proposals(dao, date):
  QUERY_VOTES = f"""query Proposals {{
  proposals(
    where: {{space_in: "{dao}", created_gt: {date}}},
    first: 1000,
    orderBy: "created",
    orderDirection: asc
  ) {{
    id
    created
    title
    votes
    type
    state
    author
    scores
    scores_total
    choices
    end
  }}
}}
"""
  return QUERY_VOTES

def set_all_proposals_info(dao):
  date = 0
  list_dic_proposals = []
  while True:
    props = requests.post(URL, json={'query': get_all_proposals(dao, date)}).json()["data"]["proposals"]

    list_dic_proposals += props
    date = max(list(map(lambda x: x["created"], props)))


    if len(props) < 1000:
      break

  df_proposals = pd.DataFrame.from_dict(list_dic_proposals, orient='columns')

  df_proposals['author'] = df_proposals['author'].str.lower()

  #comentar si se quieren las fechas de inicio y fin
  #la fecha sera la de cierre de la propuesta
  df_proposals['date'] = pd.to_datetime(df_proposals['end'], unit='s')
  #eliminamos end y created
  df_proposals = df_proposals.drop(columns=["end"])
  df_proposals = df_proposals.drop(columns=["created"])

  all_dao_info["df_proposals"] = df_proposals


set_all_proposals_info(SPACE)

In [14]:
  #añadir datos-> lista autores y votantes
list_different_authors = all_dao_info["df_proposals"]['author'].unique()
num_different_authors = len(list_different_authors)
all_dao_info["authors_list"] = list_different_authors
all_dao_info["num_authors"] = num_different_authors

list_different_voters = all_dao_info["df_votes"]['voter'].unique()
num_different_voters = len(list_different_voters)
all_dao_info["voters_list"] = list_different_voters
all_dao_info["num_voters"] = num_different_voters

In [ ]:
#followers

def query_followers(dao, skip):
  #Addresses de los followers (el número grande)
  QUERY_FOLLOWS = f"""
  query Follows {{
    follows(
      first: 20,
      skip: {skip},
      where: {{space_in: "{dao}"}}
    ) {{
      follower
    }}
  }}
  """
  return QUERY_FOLLOWS

def get_followers(dao):
  skip = 0
  list_dict_results = []
  while True:
    followers = requests.post(URL, json={'query': query_followers(dao, skip)}).json()["data"]["follows"]
    list_dict_results += followers
    skip += 20
    if len(followers) < 20:
      return list(map(operator.itemgetter('follower'), list_dict_results))


all_dao_info["basic_info"]["followers"] = get_followers(SPACE)

#print(all_dao_info["basic_info"]["followers"])

# ***BORRAR DATOS QUE NO VAMOS A USAR***

**DATOS ELIMINADOS**
En esta recogida de datos se va a forzar a:

*   Si una propuesta sigue abierta NO se analiza
*   Los votos asociados a propuestas abiertas NO se analizan



In [17]:
#BORRADO DE DATOS

# Eliminamos las propuestas que no sean "closed"
#nos quedamos con los id para borrar los votos
id_no_cerradas = all_dao_info["df_proposals"][all_dao_info["df_proposals"]["state"] != "closed"]["id"]
all_dao_info["df_proposals"] = all_dao_info["df_proposals"][all_dao_info["df_proposals"]["state"] == "closed"]
#Eliminamos los votos de las propuestas eliminadas (abiertas)
mantener = np.logical_not(np.isin(all_dao_info["df_votes"]["proposal"].apply(lambda x: x['id']) ,  id_no_cerradas))
all_dao_info['df_votes'] = all_dao_info['df_votes'][mantener]

#rearrange
all_dao_info["df_proposals"].index = range(len(all_dao_info["df_proposals"]))
all_dao_info["df_votes"].index = range(len(all_dao_info["df_votes"]))




In [18]:
  #añadir datos-> lista autores y votantes
  #actualizar el numero y la lista de autores y votantes
list_different_authors = all_dao_info["df_proposals"]['author'].unique()
num_different_authors = len(list_different_authors)
all_dao_info["authors_list"] = list_different_authors
all_dao_info["num_authors"] = num_different_authors

list_different_voters = all_dao_info["df_votes"]['voter'].unique()
num_different_voters = len(list_different_voters)
all_dao_info["voters_list"] = list_different_voters
all_dao_info["num_voters"] = num_different_voters
